<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/11_document_clustering_%E6%8E%88%E6%A5%AD%E4%B8%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング
* クラスタリングの代表的な手法であるk平均法を使ってみる。
* ついでに、言語モデルを使ったテキストマイニングを体験してみる。

## 例題: 文書クラスタリング

* Transformerベースの日本語対応言語モデルを使って、テキストのベクトル表現を得る。
  * Transformerというニューラルネットワークについては、いずれ学びます。
  * 有名な解説記事 https://jalammar.github.io/illustrated-transformer/
* テキストをベクトルとして表現することを「embedする」と言う。
  * embedすることで得られるベクトルのことを「embedding」と言う。
* そして、テキストのembeddingをk平均法でクラスタリングする。

* ランタイムのタイプをGPUにしておく。

## インストール

### spaCyの日本語モデル

* 日本語テキストを形態素解析するために使う。
  * たぶん、セッションの再起動」は不要。

In [ ]:
!python -m spacy download ja_core_news_sm

### Hugging Faceのdatasetsライブラリ

* ライブドアニュースコーパスを取得するために使う。

In [ ]:
!pip install --upgrade datasets huggingface_hub

### SentenceTransformersライブラリ
* 言語モデルを使ってテキストを埋め込む際に便利なライブラリ。
  * https://sbert.net/index.html

In [ ]:
!pip install -U sentence-transformers

## インポート

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy

from datasets import load_dataset
from transformers import set_seed
from sentence_transformers import SentenceTransformer

# 再現性の確保
set_seed(1234)

## データセット
* livedoorニュースコーパスを使う。

In [ ]:
dataset = load_dataset(
  "shunk031/livedoor-news-corpus",
  train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
  random_state=42,
  shuffle=True,
  trust_remote_code=True,
)

num_categories = len(set(dataset["train"]["category"]))

category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

print(f"num_categories: {num_categories}")
print(f"category_names: {category_names}")

In [ ]:
dataset["train"][0]

In [ ]:
dataset["train"]["title"][:10]

In [ ]:
dataset["train"]["content"][0]

## 多言語E5による埋め込み

* Multilingual E5を使う。
  * テキストのembeddingにおいて優れている言語モデル。
  * 論文 https://arxiv.org/abs/2402.05672
  * Hugging Face https://huggingface.co/intfloat/multilingual-e5-large-instruct

* 参考: テキスト埋め込みのleaderboard
  * https://huggingface.co/spaces/mteb/leaderboard

* SentenceTransformerを使ったテキストの埋め込みについては、下のWebページを参照。
  * https://sbert.net/examples/sentence_transformer/applications/computing-embeddings/README.html

In [ ]:
model_id = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(model_id)

* 試しに、一つだけ、テキストを埋め込んでみる。

In [ ]:
dataset["train"][0]["title"]

In [ ]:
model

In [ ]:
model.encode(dataset["train"][0]["title"])

* ライブドアニュースコーパスの全タイトルを埋め込む。

In [ ]:
embeddings = model.encode(dataset["train"]["title"], show_progress_bar=True)

* 埋め込みは普通にNumPyの配列として得られている。

In [ ]:
type(embeddings)

In [ ]:
embeddings.shape

* 全記事内容を埋め込むには以下のようにする。  
  * RTX3080搭載PCを使うと1分で終わる。

In [ ]:
#content_embeddings = model.encode(dataset["train"]["content"], show_progress_bar=True)

* ただし、どのテキストも先頭から512トークンで切られていることに注意。
  * 長いテキストは、途中までの内容しかembeddingに反映されない。
  * それでも、分類やクラスタリングがうまくいくことも多い。

In [ ]:
model.max_seq_length

* トークン数の調べ方
  * トークナイザにテキストを分割させる。
  * 分割によって得られたトークンの個数を数える。

In [ ]:
dataset["train"][0]["title"]

In [ ]:
model.tokenize([dataset["train"][0]["title"]])

In [ ]:
(model.tokenize([dataset["train"][0]["title"]])['input_ids']).shape[1]

* 埋め込みを保存。

In [ ]:
with open('embeddings.npy', 'wb') as f:
  np.save(f, embeddings)

In [ ]:
#with open('content_embeddings.npy', 'wb') as f:
#  np.save(f, content_embeddings)

* 読み込みは以下のようにする。

In [ ]:
with open('embeddings.npy', 'rb') as f:
  embeddings = np.load(f)

In [ ]:
#with open('content_embeddings.npy', 'rb') as f:
#  content_embeddings = np.load(f)

## クラスタのラベリングに使う単語の抽出

* 全テキストを形態素解析する。
  * 形態素解析＝単語への分割

In [ ]:
nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(dataset["train"]["title"]):
  corpus.append(" ".join([token.lemma_ for token in nlp(text)]))

In [ ]:
dataset["train"][0]["title"]

In [ ]:
corpus[0]

* scikit-learnでTF-IDFを計算する。
* `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [ ]:
vectorizer = TfidfVectorizer(min_df=20)

In [ ]:
X_train = vectorizer.fit_transform(corpus)

In [ ]:
X_train = X_train.toarray()

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
vocab = np.array(vectorizer.get_feature_names_out())

In [ ]:
vocab.size

In [ ]:
print(list(vocab))

## ラベリング用単語の埋め込み

* 各単語について、その単語を含むテキストの埋め込みベクトルの加重平均を求める。
* 加重平均の重みは、各テキストにおけるその単語のTF-IDFの値を使って定める。

In [ ]:
X_train.sum(0)

In [ ]:
text_weights = X_train / X_train.sum(0)

In [ ]:
vocab_embeddings = np.dot(text_weights.T, embeddings)

## 文書クラスタリング



In [ ]:
embeddings.shape

* k-meansのしくみ

* 初期化

In [ ]:
n_clusters = 20
assignments = np.random.randint(0, n_clusters, X_train.shape[0])

In [ ]:
assignments

* クラスタの重心の計算

In [ ]:
mean_vectors = []
for k in range(n_clusters):
  mean_vectors.append(embeddings[assignments == k].mean(0))

In [ ]:
mean_vectors = np.array(mean_vectors)

In [ ]:
mean_vectors

* 各ベクトルに最も近い重心ベクトルを見つけて、クラスタを割り当て直す

In [ ]:
np.sqrt(((embeddings[0] - mean_vectors[0]) ** 2).sum())

In [ ]:
distances = []
for k in range(n_clusters):
  distances.append(np.linalg.norm(embeddings[0] - mean_vectors[k]))
distances

### k-平均法によるクラスタリング

In [ ]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
#kmeans.fit(content_embeddings) # 本文の場合はこちら。
centers = kmeans.cluster_centers_

In [ ]:
centers

* クラスタの重心を保存。

In [ ]:
with open(f'centers_{n_clusters}.npy', 'wb') as f:
  np.save(f, centers)

In [ ]:
with open(f'centers_{n_clusters}.npy', 'rb') as f:
  centers = np.load(f)

### クラスタのサイズを調べる

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
print([sorted(size_dict.items(), key=lambda item: item[1], reverse=True)])

## クラスタのラベリング
* 各クラスタの重心に近い単語でラベリングする。

* テキストの埋め込みは、長さ1のベクトルになっている。

In [ ]:
np.linalg.norm(embeddings, axis=-1)

* テキストとラベリング用の単語との類似度はコサイン類似度で測る。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vocab_embeddings, centers)

In [ ]:
vocab_embeddings.shape

In [ ]:
centers.shape

In [ ]:
similarities.shape

* 重心に近い順に30個の単語を表示する。

In [ ]:
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  print(vocab[indices[:30]])

# プランナー課題１１
* それぞれのクラスタについて、重心に近い元々のテキスト（つまり記事タイトル）を5件ずつ表示させてみよう。
* それらのテキストの内容に、上で得たラベルが合っているかどうか、確かめよう。